In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys, time, glob, pickle
from tqdm import tqdm
from utils_opsim import *
#lsst modules
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.db as db
import lsst.sims.maf.plots as plots
import lsst.sims.maf.slicers as slicers
from lsst.sims.maf.metrics import BaseMetric
import lsst.sims.maf.metricBundles as metricBundles
from lsst.sims.photUtils import Dust_values
from lsst.sims.photUtils import Bandpass, SignalToNoise, PhotometricParameters, calcMagError_m5, calcGamma
from lsst.sims.utils import uniformSphere

/data0/lsst_stack/stack/miniconda3-py38_4.9.2-0.4.2/Linux64/sims_maf/2.13.0.sims-93-g8bc2eb33+a14e41332a/python/lsst/sims/maf/db/opsimDatabase.py:95: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if groupBy is 'default' and tableName==self.defaultTable:
/data0/lsst_stack/stack/miniconda3-py38_4.9.2-0.4.2/Linux64/sims_maf/2.13.0.sims-93-g8bc2eb33+a14e41332a/python/lsst/sims/maf/db/opsimDatabase.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if groupBy is 'default' and tableName!=self.defaultTable:


In [ ]:
import zipfile

In [ ]:
#This cell is needed only the very first time when the simulated lightcurves have to be imported on the platform
#with zipfile.ZipFile('./True Novelties/kn_newmodel_Alberto.zip', "r") as z:
#      z.extractall("./")

In [5]:
lc = glob.glob('./kn_newmodel_Alberto/*')

In [2]:
class GRBKN_obs(metrics.BaseMetric):
    """
        Class object, observed lightcurve from LSST observing constraints
        Parameters
        ------------
            mjdCol= MJD observations column name from Opsim database      (DEFAULT = observationStartMJD) 
            m5Col= Magnitude limit column name from Opsim database      (DEFAULT = fiveSigmaDepth)
            filterCol= Filters column name from Opsim database      (DEFAULT = filter)
            exptimeCol = Column name for the total exposure time of the visit(DEFAULT = visitExposureTime)
            nightCol = The night's column of the survey (starting at 1) (DEFAULT = night)
            vistimeCol = Column name for the total time of the visit (DEFAULT = visitTime)
            RACol= RA column name from Opsim database      (DEFAULT = fieldRA)
            DecCol= Dec column name from Opsim database      (DEFAULT = fieldDec)
            surveyduration= Survey Duration      (DEFAULT = 10)
            mjd0= Survey start date      (DEFAULT = 59853.5)
            Filter_selection = 
        Returns
        -------
            nobj: number of detected lightcurves
            csv files with the observed lightcurves
        """
    def read_lightCurve(self, file):
        """Reads in a csv file, from the simulated ligh curves, time and mag columns for each filter
        Returns
        -------
        numpy.ndarray
            The data read from the ascii text file, in a numpy structured array with columns
            'ph' (phase / epoch, in days), 'mag' (magnitude), 'flt' (filter for the magnitude).
        """
        time = np.arange(0.5, 20 + 0.5, 0.5)
        ll = pd.DataFrame(file, columns=['file_paths'])
        ll['lc']=ll['file_paths'].apply(lambda x: pd.read_csv(x, index_col=False))
        ll['transDuration'] = ll['lc'].apply(lambda x: time.max() - time.min())
        ll['T0'] = ll['lc'].apply(lambda x: time.min())
        return ll
    
    def __init__(self, metricName='GRBKN_obs', mjdCol='observationStartMJD', 
                 RACol='fieldRA', DecCol='fieldDec',filterCol='filter', m5Col='fiveSigmaDepth', 
                 exptimeCol='visitExposureTime',nightCol='night',vistimeCol='visitTime', snrlim=5, ptsNeeded=2, dist_Mpc=46.48, mjd0=59853.5,
                 data_path='./lc', obs_path='./obs_lc',surveyduration=10,Filter_selection = False,nFilter=1, save=True,**kwargs):
        maps = ['DustMap']
        self.mjdCol = mjdCol
        self.m5Col = m5Col
        self.filterCol = filterCol
        self.RACol = RACol
        self.DecCol = DecCol
        self.exptimeCol = exptimeCol
        self.nightCol = nightCol
        self.vistimeCol = vistimeCol
        self.nightCol = nightCol
        self.ptsNeeded = ptsNeeded
        self.data_path = data_path
        self.obs_path = obs_path
        self.dist_Mpc = dist_Mpc
        self.surveyduration=surveyduration
        self.mjd0=mjd0
        self.Filter_selection=Filter_selection
        self.nFilter=nFilter
        self.snrlim = snrlim
        self.save = save
        if not os.path.exists(self.obs_path):
            os.mkdir(self.obs_path)
        self.bandpass = Bandpass(wavelen=np.array([480.2,623.1,754.2]),
                                 sb=np.array([0.1,0.1,0.1]),wavelen_min=380, wavelen_max=850, wavelen_step=10)
        super(GRBKN_obs, self).__init__(col=[self.mjdCol,self.m5Col, self.filterCol,self.RACol,
                                                                   self.DecCol,self.exptimeCol,self.nightCol,
                                                                   self.vistimeCol],
                                                       metricDtype='object', units='',
                                                       metricName=metricName, **kwargs)
        self.bandpass = Bandpass(wavelen=np.array([480.2,623.1,754.2]),sb=np.array([0.1,0.1,0.1]),wavelen_min=380, wavelen_max=850, wavelen_step=10)
        self.photparam = PhotometricParameters() 
        lcfile = glob.glob(self.data_path+'/*.csv')
        lcfile = np.array(self.lcfile).reshape(-1)
        self.lcs = self.read_lightCurve(flc)
     
    def make_lightCurve(self, template, time, filters,transduration):
        """Turn lightcurve definition into magnitudes at a series of times.
        Parameters
        ----------
        time : numpy.ndarray
            The times of the observations.
        filters : numpy.ndarray
            The filters of the observations.
        Returns
        -------
        numpy.ndarray
             The magnitudes of the transient at the times and in the filters of the observations.
        """
        flt = np.unique(filters)
        lcMags = np.zeros(time.size, dtype=float)
        time = (time-self.mjd0)%transduration
        for key in set(flt):
            # Interpolate the lightcurve template to the times of the observations, in this filter.
            temp_ph=np.array(np.logspace(-1,np.log10(30),50),float)#template['time'], float)
            lcMags[filters==key] = np.interp(time[filters==key], temp_ph,
                                        np.array(template[key], float))
        return lcMags
    def coadd(self, data):
        """
        Method to coadd data per band and per night
        Parameters
        ------------
        data : `pd.DataFrame`
            pandas df of observations
        Returns
        -------
        coadded data : `pd.DataFrame`
        """

        keygroup = [self.filterCol, self.nightCol]

        data.sort_values(by=keygroup, ascending=[
                         True, True], inplace=True)

        coadd_df = data.groupby(keygroup).agg({self.vistimeCol: ['sum'],
                                               self.exptimeCol: ['sum'],
                                               self.mjdCol: ['mean'],
                                               self.RACol: ['mean'],
                                               self.DecCol: ['mean'],
                                               self.m5Col: ['mean']}).reset_index()

        coadd_df.columns = [self.filterCol, self.nightCol, 
                            self.vistimeCol, self.exptimeCol, self.mjdCol,
                            self.RACol, self.DecCol, self.m5Col]

        coadd_df.loc[:, self.m5Col] += 1.25 * \
            np.log10(coadd_df[self.vistimeCol]/30.)

        coadd_df.sort_values(by=[self.filterCol, self.nightCol], ascending=[
                             True, True], inplace=True)

        return coadd_df.to_records(index=False)

    def name_lc(self, t, lcname, name):
        return self.obs_path+'/obs_{}_'.format(name)+lcname+'_T={}.npy'.format(t)
    def run(self,dataSlice, slicePoint=None):
        # Sort the entire dataSlice in order of time. 
        dataSlice.sort(order=self.mjdCol)
        dataSlice = self.coadd(pd.DataFrame(dataSlice))
        
        field_idx = np.random.choice(range(np.size(dataSlice[self.RACol])))
        Ra, Dec = dataSlice[self.RACol][field_idx],dataSlice[self.DecCol][field_idx]
        bandpass = self.bandpass
        photparam = self.photparam
        calcSNR_m5=np.vectorize(SignalToNoise.calcSNR_m5)
        
        obs_filter = dataSlice[self.filterCol]
        obs = dataSlice[self.mjdCol]       
        obs_m5 = dataSlice[self.m5Col]
        nobj = np.array([0,0],[('detected','i4'),('undetected','i4')]) 
        

        lcNumberStart = -1 * np.floor((dataSlice[self.mjdCol].min() - self.mjd0) / self.lcs['transDuration'].to_numpy()[:,None])
        # Calculate the time/epoch for each lightcurve.
        lcEpoch = (obs-self.mjd0) % self.lcs['transDuration'].to_numpy()[:,None]
        # Identify the observations which belong to each distinct light curve.
        lcNumber = np.floor((obs-self.mjd0 )  / lcs['transDuration'].to_numpy()[:,None]) + lcNumberStart
        lcNumberStart = lcNumber.max(axis=0)
        ulcNumber = np.array(list(map(np.unique,lcNumber)))
        lcLeft = np.array([np.searchsorted(lcNumber[i,:], ulcNumber[i,:], side='left')for i in range(ulcNumber.shape[0])])
        lcRight = np.array([np.searchsorted(lcNumber[i,:], ulcNumber[i,:], side='right')for i in range(ulcNumber.shape[0])])
        self.lcs['obs'] = self.lcs.apply(lambda x: self.make_lightCurve(x['lc'],obs, obs_filter,x['transDuration']), axis=1)
        lcMag = np.array(self.lcs['obs'].tolist())  + 5*np.log10(self.dist_Mpc*10**6)-5
        gamma = np.array([calcGamma(self.bandpass, m, self.photparam) 
                          for m in obs_m5])

        merr, _= calcMagError_m5(lcMag, bandpass, obs_m5,
                                    photparam, gamma=gamma)
        #lcSNR,_ = calcSNR_m5(lcMag, bandpass, obs_m5, photparam)
        lcpoints_AboveThresh = np.zeros(np.shape(lcMag), dtype=bool) 
        for f in np.unique(obs_filter):                    
            filtermatch = np.where(obs_filter == f)
            lcpoints_AboveThresh[:,filtermatch] = np.where(lcMag[:,filtermatch] <= obs_m5[filtermatch],True,lcpoints_AboveThresh[:,filtermatch])
        T0 = lcs['T0'][:,None] + self.mjd0 + ulcNumber*lcs['transDuration'].to_numpy()[:,None]

        #INITIALIZATION 10yrs LCs
        detection_map = np.array([[lcpoints_AboveThresh[i,init:end] for j,(init,end) in enumerate(zip(lcLeft[i,:],lcRight[i,:])) if init!= end]for i in range(len(flc))],dtype=object)
        lcEpochs= np.array([[lcEpoch[i,init:end] for j,(init,end) in enumerate(zip(lcLeft[i,:],lcRight[i,:])) if init!= end]for i in range(len(flc)) ],dtype=object)
        lcMags = np.array([[lcMag[i,init:end] for j,(init,end) in enumerate(zip(lcLeft[i,:],lcRight[i,:])) if init!= end]for i in range(len(flc)) ],dtype=object)
        lcMerr = np.array([[merr[i,init:end] for j,(init,end) in enumerate(zip(lcLeft[i,:],lcRight[i,:])) if init!= end]for i in range(len(flc)) ],dtype=object)
        lcfilters = np.array([[obs_filter[init:end] for j,(init,end) in enumerate(zip(lcLeft[i,:],lcRight[i,:])) if init!= end]for i in range(len(flc)) ],dtype=object)
        lcmaglim = np.array([[obs_m5[init:end] for j,(init,end) in enumerate(zip(lcLeft[i,:],lcRight[i,:])) if init!= end]for i in range(len(flc)) ],dtype=object)

        Dpoints = np.array([[np.sum(detections, axis= 0) for detections in detection_map[i]] for i in range(len(flc))])  #counts the number of detected points
        nobj['detected']=np.sum([points>self.ptsNeeded for i in range(len(flc))  for points in Dpoints[i] ])
        nobj['undetected']=np.sum([points>self.ptsNeeded for i in range(len(flc))  for points in Dpoints[i] ])
        if self.save:
            shift = np.array([[ulcNumber[i,j]*lcs['transDuration'][i] for j,(init,end) in enumerate(zip(lcLeft[i,:],lcRight[i,:])) if init!= end]for i in range(len(flc)) ])
            epochs = lcEpochs +  self.mjd0 + shift

            total_lc = np.dstack([epochs,
                    lcfilters,
                    lcMags,
                    lcMerr,
                    lcmaglim,
                    detection_map])

            name_ = np.vectorize(self.name_lc)
            names = name_(T0,
                          np.repeat(np.array(flc)[:,None],ulcNumber.shape[1],axis=0).reshape(ulcNumber.shape),
                          ulcNumber)
            names = np.expand_dims(names, axis=-1)
            out = list(zip(names,total_lc))
            with open(self.obs_path+'/simRA={}_DEC={}_part{}.pkl'.format(np.round(Ra,2),np.round(Dec,2), nflc+1), 'wb') as outfile:
                pickle.dump(out, outfile)
            ndet = np.sum(Dpoints)
            res = ndet/nobj['detected'][0]/6
            print(res)
        else:
            ndet = np.sum(Dpoints)
            res = ndet/nobj['detected'][0]/6
            print(res)
        return res
           

In [3]:
fbs = glob.glob('/sims_maf/fbs_2.0/*')
dbDir = '/sims_maf/fbs_2.0/*'
outDir = './nmma_lsst_outputv2.00_newmodel_pop'
resultsDb = db.ResultsDb(outDir=outDir)
db_folders=glob.glob(dbDir+'*')

In [4]:
folder_f= np.vectorize(os.path.isdir)
glob_f = np.vectorize(glob.glob)

In [5]:
dbases = []
for folder in db_folders:
     if os.path.isdir(folder):
        for sub in glob.glob(folder+'/*'):
            if os.path.isdir(sub):
                for subsub in glob.glob(sub+'/*.db'):
                    if 'v2.0_10yrs.db' in subsub:
                        dbases.append(subsub)
            elif 'v2.0_10yrs.db' in sub:
                dbases.append(sub)


In [6]:
def generateKNPopSlicer( n_events=10000, seed=42):
    """ Generate a population of KNe events, and put the info about them
    into a UserPointSlicer object
    Parameters
    ----------
    n_events : int (10000)
        The number of kilonova events to generate
    seed : float
        The seed passed to np.random
    """

    def rndm(a, b, g, size=1):
        """Power-law gen for pdf(x)\propto x^{g-1} for a<=x<=b"""
        r = np.random.random(size=size)
        ag, bg = a**g, b**g
        return (ag + (bg - ag)*r)**(1./g)

    ra, dec = uniformSphere(n_events, seed=seed)
    
    # Define the distance
    
    # Set up the slicer to evaluate the catalog we just made
    slicer = slicers.UserPointsSlicer(ra, dec, latLonDeg=True, badval=0)
    return slicer

In [7]:
metric42 = GRBKN_obs(metricName='GRBKN_obs42', data_path='./kn_newmodel_Alberto',obs_path='./obs_newmodel_Alberto_42', dist_Mpc = 42,save=False)
metric100 = GRBKN_obs(metricName='GRBKN_obs100', data_path='./kn_newmodel_Alberto',obs_path='./obs_newmodel_Alberto_100', dist_Mpc = 100,save=False)
metric300 = GRBKN_obs(metricName='GRBKN_obs300', data_path='./kn_newmodel_Alberto',obs_path='./obs_newmodel_Alberto_300', dist_Mpc = 300,save=False)

# bundle
sqlconstraint='note not like  "DD%"'
# this is for the "Targeted Search"
## coo={'ra':np.array([197.450341598]),'dec':np.array([-23.3814675445])}
# this is for the "All Sky Search"
slicer = generateKNPopSlicer(n_events = 500)
Sky42 = metricBundles.MetricBundle(metric42, slicer, sqlconstraint)
Sky100 = metricBundles.MetricBundle(metric100, slicer, sqlconstraint)
Sky300 = metricBundles.MetricBundle(metric300, slicer, sqlconstraint)
# group bundle
bundleDict ={'GRBKN_obs42':Sky42,'GRBKN_obs100':Sky100,'GRBKN_obs300':Sky300}

In [8]:
from tqdm import tqdm

In [9]:
start = time.time()
for dbs in tqdm(dbases):
    opsdb = db.OpsimDatabase(dbs)
    #for k, (sim, obs) in enumerate(zip([],['./obs_newmodel_Alberto'])): #enumerate(zip(['./sim_knlc_46p48Mpc','./sim_knlc_100Mpc','./sim_knlc_300Mpc'],['./obs_lc_wfd_knlcv2.99_46p48Mpc_noDD','./obs_lc_wfd_knlcv2.99_100Mpc_noDD','./obs_lc_wfd_knlcv2.99_300Mpc_noDD'])):        
    Sky42.setRunName(dbs)
    Sky100.setRunName(dbs)
    Sky300.setRunName(dbs)
    group = metricBundles.MetricBundleGroup(bundleDict, opsdb, outDir = outDir, resultsDb=resultsDb, dbTable='observations',)
    # run
    group.runAll()
t = time.time()-start
print('time:{} min'.format(t/60))

  0%|          | 0/126 [00:00<?, ?it/s]/data0/lsst_stack/stack/miniconda3-py38_4.9.2-0.4.2/Linux64/sims_maf/2.13.0.sims-93-g8bc2eb33+a14e41332a/python/lsst/sims/maf/db/opsimDatabase.py:46: UserWarning: Could not identify opsim database version; just using Database class instead
  warnings.warn('Could not identify opsim database version; just using Database class instead')


Querying database observations with constraint note not like  "DD%" for columns ['observationStartMJD', 'fiveSigmaDepth', 'fieldRA', 'fieldDec', 'night', 'visitTime', 'visitExposureTime', 'filter']
Found 1992714 visits
Running:  ['GRBKN_obs42']
0


<ipython-input-2-3c6abeebe167>:175: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  T0 = lcs['T0'][:,None] + self.mjd0 + ulcNumber*lcs['transDuration'].to_numpy()[:,None]


6.375766293488209
0
7.681459069876592
0
5.34965153712658
0
4.7027541960461
0
4.378141321834437
0
5.611488510831731
0
8.606685625251965
0
7.4549151772282505
0
6.754345407471848
0


<ipython-input-2-3c6abeebe167>:212: RuntimeWarning: divide by zero encountered in long_scalars
  res = ndet/nobj['detected'][0]/6


inf
0


  0%|          | 0/126 [50:36<?, ?it/s]
